In [30]:
import sqlalchemy

In [31]:
sqlalchemy.__version__
## dialect + driver :// username : password@ host:port/databes
   ## 커넥터  드라이버 sqllite  sqlite는 서버없어서 그냥 밑에처럼 간단함

'1.2.1'

In [32]:
from sqlalchemy import create_engine

In [33]:
engine = create_engine('sqlite:///test.db', echo = True)  ## 엔진을 통해 어떤 작업할떄 내가한 작업에 대한 에코

In [34]:

print(engine)

Engine(sqlite:///test.db)


In [35]:
type(engine)

sqlalchemy.engine.base.Engine

In [36]:
## 메타데이터 >> 테이블 >> 컬럼  ## 메타데이터가 제일큼

In [37]:
from sqlalchemy import *

In [38]:
metadata = MetaData()

In [39]:
users = Table('users', metadata, 
             Column('id', Integer, primary_key=True),
             Column('name', String),
             Column('fullname', String),
)


In [40]:
addresses = Table('addresses', metadata, 
                 Column('id', Integer, primary_key=True),
                 Column('user_id', None, ForeignKey('users.id')),
                 Column('email_address', String, nullable=False)
    )

In [41]:
metadata.tables

immutabledict({'users': Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None), 'addresses': Table('addresses', MetaData(bind=None), Column('id', Integer(), table=<addresses>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.id'), table=<addresses>), Column('email_address', String(), table=<addresses>, nullable=False), schema=None)})

In [42]:
metadata.bind

In [43]:
metadata.create_all(engine) ## immutable alter update 이런거 안된다. 지워서 다시 엔진에 등록시켜야함  
                            ## commit까지 된다. ## sql 에 join on 절에 안써도 알아서 조인이 된다.


2019-07-09 11:19:43,285 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 11:19:43,286 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:19:43,288 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 11:19:43,290 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:19:43,292 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 11:19:43,293 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:19:43,295 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-07-09 11:19:43,296 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:19:43,297 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 11:19:43,298 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:19:43,320 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 11:19:43,321 INFO sqlalchemy.engine.b

In [ ]:
### metadata.bind

In [44]:
insert = users.insert()
print(insert)

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [45]:
insert = users.insert().values(name = 'kim', fullname = 'Anonymous, Kim')
print(insert)


INSERT INTO users (name, fullname) VALUES (:name, :fullname)


In [47]:
insert.compile().params   ##sql문을 자동으로 완성시켜줌 안쳐도됨, name, fullname이란 키값에 데이터 박아넣음

{'fullname': 'Anonymous, Kim', 'name': 'kim'}

In [48]:
##resultproxy --> cursor랑 똑같다.

In [49]:
conn = engine.connect()
conn

In [50]:
insert.bind = engine

In [51]:
str(insert)

'INSERT INTO users (name, fullname) VALUES (?, ?)'

In [52]:
result = conn.execute(insert)  ##Insert에서 만들어진 sql문을 실행 result 커서랑 같은거

2019-07-09 11:31:50,022 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:31:50,023 INFO sqlalchemy.engine.base.Engine ('kim', 'Anonymous, Kim')
2019-07-09 11:31:50,027 INFO sqlalchemy.engine.base.Engine COMMIT


In [53]:
##가장 최근 작업한 id 여기서는 1이 나와야한다
result.inserted_primary_key

[1]

In [54]:
##################### 앞에 복잡한과정 필요없이 이렇게 간단하게 insert 할수있따.
insert = users.insert()
result = conn.execute(insert, name="lee", fullname = "Unknoewn, Lee")
result.inserted_primary_key


2019-07-09 11:33:33,269 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:33:33,270 INFO sqlalchemy.engine.base.Engine ('lee', 'Unknoewn, Lee')
2019-07-09 11:33:33,274 INFO sqlalchemy.engine.base.Engine COMMIT


[2]

In [57]:
## 파라메터 여러개 == 쿼리 여러개 한번에 넣는거  commit 까지 다된다.
conn.execute(addresses.insert(), [
    {"user_id":1, "email_address":"anonymous.kim@"},
    {"user_id":2, "email_address":"unknown.lee@"}
])

2019-07-09 11:41:32,476 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2019-07-09 11:41:32,478 INFO sqlalchemy.engine.base.Engine ((1, 'anonymous.kim@'), (2, 'unknown.lee@'))
2019-07-09 11:41:32,482 INFO sqlalchemy.engine.base.Engine COMMIT


In [58]:
##########################################################

In [59]:
##select

In [66]:
query = select([users])## 다가져옴 select 쿼리


In [67]:
users

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None)

In [68]:
cur = conn.execute(query)

2019-07-09 11:46:03,292 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 11:46:03,293 INFO sqlalchemy.engine.base.Engine ()


In [70]:
for row in cur :
    print (row)

In [69]:
cur.fetchall()

[(1, 'kim', 'Anonymous, Kim'), (2, 'lee', 'Unknoewn, Lee')]